In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F


from torch.utils.data import Dataset, DataLoader
import numpy as np
import struct
import os
from torchvision import transforms 
from PIL import Image,ImageDraw
import torch.optim as optim


import tkinter as tk



In [5]:

class obrazki(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),     
            nn.ReLU(),              
            nn.MaxPool2d(2),        # Zmniejsza obrazek 2x (z 28x28 na 14x14)
            
            # Blok 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)         # Zmniejsza obrazek 2x (z 14x14 na 7x7)
        )
        
       
        self.flatten = nn.Flatten()
        
  
        self.classifier = nn.Sequential(
            nn.Linear(64 * 7 * 7, 512), 
            nn.ReLU(),
            nn.Dropout(0.5),            
            
            
            nn.Linear(512, 128),        
            nn.ReLU(),
            nn.Dropout(0.5),
            
            nn.Linear(128, 10)# Wyjście: 10 klas (cyfry MNIST)
        )

    def forward(self, x):

        x = self.encoder(x)

        x = self.flatten(x)

        x = self.classifier(x)
        
        return x 

In [6]:
class AplikacjaAI:
    def __init__(self, root):
        self.root = root
        self.root.title("Rysowanie Cyfr dla AI")


        self.CANVAS_SIZE = 600  
        self.BRUSH_SIZE = 30    
    

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if torch.backends.mps.is_available(): self.device = torch.device("mps")
        
        self.model = obrazki().to(self.device)
        try:
            self.model.load_state_dict(torch.load('moj_drugi_model.pth', map_location=self.device))
            self.model.eval()
            print("Model załadowany pomyślnie!")
        except Exception as e:
            print(f"Błąd modelu: {e}")

        self.canvas = tk.Canvas(root, width=self.CANVAS_SIZE, height=self.CANVAS_SIZE, bg='black', cursor="cross")
        self.canvas.pack(pady=10)

        self.image = Image.new("L", (self.CANVAS_SIZE, self.CANVAS_SIZE), 0)
        self.draw = ImageDraw.Draw(self.image)


        self.canvas.bind("<B1-Motion>", self.rysuj) 

   
        self.btn_predict = tk.Button(root, text="ZGADNIJ CYFRĘ", font=("Arial", 14), command=self.zgadnij)
        self.btn_predict.pack(pady=5)

 
        self.btn_clear = tk.Button(root, text="Wyczyść", command=self.wyczysc)
        self.btn_clear.pack(pady=5)


        self.label_result = tk.Label(root, text="Narysuj cyfrę...", font=("Arial", 20))
        self.label_result.pack(pady=10)

    def rysuj(self, event):
        # Rysowanie kółek w miejscu kursora (tworzy linię)
        x1, y1 = (event.x - self.BRUSH_SIZE), (event.y - self.BRUSH_SIZE)
        x2, y2 = (event.x + self.BRUSH_SIZE), (event.y + self.BRUSH_SIZE)
        
        # 1. Rysuj na ekranie (dla Ciebie) - biały kolor
        self.canvas.create_oval(x1, y1, x2, y2, fill="white", outline="white")
        
        # 2. Rysuj w pamięci (dla AI) - biały kolor (255)
        self.draw.ellipse([x1, y1, x2, y2], fill=255, outline=255)

    def wyczysc(self):

        self.canvas.delete("all")

        self.image = Image.new("L", (self.CANVAS_SIZE, self.CANVAS_SIZE), 0)
        self.draw = ImageDraw.Draw(self.image)
        self.label_result.config(text="Narysuj cyfrę...")

    def zgadnij(self):
  
        img_resized = self.image.resize((28, 28), Image.BILINEAR)

        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])
        
    
        img_tensor = transform(img_resized).unsqueeze(0).to(self.device)


        with torch.no_grad():
            output = self.model(img_tensor)
            probs = F.softmax(output, dim=1)
            pewnosc, indeks = torch.max(probs, 1)
            
        wynik = indeks.item()
        procent = pewnosc.item() * 100
        
    
        self.label_result.config(text=f"Widzę cyfrę: {wynik}\nPewność: {procent:.1f}%")

# URUCHOMIENIE APLIKACJI
if __name__ == "__main__":
    root = tk.Tk()
    app = AplikacjaAI(root)
    root.mainloop()

Model załadowany pomyślnie!
